# pinn class

In [ ]:
import torch
import torch.nn as nn
import numpy as np


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, lambda2):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        self.lambda2 = lambda2

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred,
            g,
            torch.ones([xtrain.shape[0], 1]).to("cuda"),
            retain_graph=True,
            create_graph=True,
        )[0]
        u_xx_tt = torch.autograd.grad(
            u_x_t, g, torch.ones(xtrain.shape).to("cuda"), create_graph=True
        )[0]
        u_x = u_x_t[:, [0]]
        u_t = u_x_t[:, [1]]
        u_xx = u_xx_tt[:, [0]]
        return self.loss(u_t + (u_pred * u_x) - (self.lambda2 * u_xx), fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}")
        return loss.item()

# load data

In [8]:
from math import ceil
dt_save = 0.01
ini_time = 0.0
fin_time = 2.0
nx = 1024
xL = -1.0
xR = 1.0
if_second_order = 1.0
show_steps = 100
dx = (xR - xL) / nx
xe = np.linspace(xL, xR, nx + 1)
x = xe[:-1] + 0.5 * dx
it_tot = ceil((fin_time - ini_time) / dt_save) + 1
t = np.arange(it_tot + 1) * dt_save
x.shape, t.shape

((1024,), (202,))

In [25]:
X, T = np.meshgrid(x, t[:-1])
xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
xtrue = torch.tensor(xtrue, dtype=torch.float32, device="cuda")
device = "cuda"

In [ ]:
import torch
import numpy as np
def loadAndPrep(u):
    idx = np.random.choice(u.flatten().shape[0], 10000, replace=False)
    xtrain = xtrue[idx, :]
    utrain = u.flatten()[idx][:, None]
    utrain = torch.tensor(utrain, dtype=torch.float32).to(device)
    return xtrain, utrain

In [27]:
import gc
def trainAndLog(u, e):
    xtrain, utrain = loadAndPrep(u)
    model = PINN(input_size=2, hidden_size=20, output_size=1, lambda2=e).to("cuda")
    loss = model.train_model(xtrain, utrain, epochs=5000)
    with torch.no_grad():
        pred = model(xtrue).cpu().numpy()
    l = np.mean((u.flatten() - pred.flatten())**2)
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return l, loss

In [ ]:
import h5py
import random
import json

random.seed(69)
d = {}
# /kaggle/input/burgers-clean/simulation_data.h5
with h5py.File("simulation_data.h5", "r") as f:
    a = random.choices(list(f.keys()), k=50)
    n = 0
    for i in a:
        print(n)
        e = f[i]["epsilon"][()]
        print(e / np.pi)
        uclean = f[i]["clean"][:]
        mse, loss = trainAndLog(uclean, e)
        unoisy = f[i]["noisy"][:]
        mse1, loss1 = trainAndLog(unoisy, e)
        d[i] = {
            "clean": {"mse": float(mse), "loss": float(loss)},
            "noisy": {"mse": float(mse1), "loss": float(loss1)},
        }
        n += 1
        with open("results.json", "w") as g:
            json.dump(d, g)